# Cắt ảnh khuôn mặt từ một tấm ảnh

In [ ]:
import os
import cv2
import face_recognition

# Đường dẫn tới thư mục chứa ảnh đầu vào và đầu ra
input_dir = "ToolPreprocessingData\\DataInput"
output_dir = "ToolPreprocessingData\\DataOutput"

# Đảm bảo thư mục đầu ra tồn tại
os.makedirs(output_dir, exist_ok=True)


# Hàm để phát hiện và cắt khuôn mặt, rồi thay đổi kích thước ảnh
def detect_and_resize_face(img_path, output_path):
    img = face_recognition.load_image_file(img_path)
    face_locations = face_recognition.face_locations(img)
    if face_locations:
        top, right, bottom, left = face_locations[
            0
        ]  # Chỉ xử lý khuôn mặt đầu tiên phát hiện được
        face = img[top:bottom, left:right]
        gray_face = cv2.cvtColor(face, cv2.COLOR_RGB2GRAY)  # Chuyển sang thang độ xám
        resized_face = cv2.resize(gray_face, (48, 48))  # Thay đổi kích thước ảnh
        cv2.imwrite(output_path, resized_face)  # Lưu ảnh đã xử lý


# Xử lý tất cả các ảnh trong thư mục đầu vào
for img_name in os.listdir(input_dir):
    input_path = os.path.join(input_dir, img_name)
    output_path = os.path.join(output_dir, img_name)
    detect_and_resize_face(input_path, output_path)

print("Hoàn thành tiền xử lý ảnh.")

# Xử lý dữ liệu

In [ ]:
import os
import csv
from PIL import Image
import numpy as np


def image_to_pixels(image_path):
    img = Image.open(image_path)

    # Kiểm tra xem ảnh có phải là ảnh xám không
    if img.mode != "L":
        img = img.convert("L")  # Chuyển đổi ảnh sang thang độ xám nếu cần

    pixels = np.array(img).flatten()  # Chuyển đổi ảnh thành mảng một chiều
    return pixels


def image_to_data_csv(image_folder_path, output_directory):
    emotion_map = {
        "angry": 0,
        "disgusted": 1,
        "fearful": 2,
        "happy": 3,
        "neutral": 4,
        "sad": 5,
        "surprised": 6,
    }
    for image_file in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_file)
        if os.path.isfile(image_path) and image_file.lower().endswith(
            (".png", ".jpg", ".jpeg")
        ):
            parts = image_folder_path.split(os.sep)
            emotion = parts[-1]  # Thư mục cảm xúc
            usage = parts[-2]  # Thư mục sử dụng (Train/Val/Test)

            if emotion not in emotion_map:
                continue 

            emotion_value = emotion_map[emotion]

            # Lấy giá trị pixel chuẩn hóa từ ảnh
            pixels = image_to_pixels(image_path)

            os.makedirs(output_directory, exist_ok=True)

            csv_path = os.path.join(output_directory, "output.csv")
            file_exists = os.path.isfile(csv_path)
            with open(csv_path, "a", newline="") as file:
                writer = csv.writer(file)
                if not file_exists:
                    writer.writerow(["emotion", "pixels", "Usage"])
                writer.writerow([emotion_value, " ".join(map(str, pixels)), usage])
    print("Đã xong!")

image_to_data_csv("emodtcsv\\train\\fearful", "data_created")
image_to_data_csv("emodtcsv\\train\\happy", "data_created")
image_to_data_csv("emodtcsv\\train\\neutral", "data_created")
image_to_data_csv("emodtcsv\\train\\sad", "data_created")
image_to_data_csv("emodtcsv\\train\\surprised", "data_created")
